In [1]:
# Model without CRF

from keras.utils.np_utils import to_categorical
#from tflearn.data_utils import to_categorical

C:\Users\hp\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.models import load_model

In [3]:
import pandas as pd
import numpy as np
from keras.models import load_model


data = pd.read_csv('NERData.csv', encoding='latin1', sep = ',')
data = data.fillna(method="ffill")
words = list(set(data["Word"].values))
words.append("ENDPAD")
n_words = len(words)
tags = list(set(data["Tag"].values))
n_tags = len(tags)

class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

getter = SentenceGetter(data)
sent = getter.get_next()
sentences = getter.sentences
max_len = 50
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words - 1)
y = [[tag2idx[w[2]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])
#from keras.utils import to_categorical
y = [to_categorical(i, num_classes=n_tags) for i in y]
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words, output_dim=50, input_length=max_len)(input)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(model)  # softmax output layer
model = Model(input, out)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

In [4]:
import keras

In [5]:
import tensorflow as tf

In [6]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=100, validation_split=0.1, verbose=1)

#model.save('mmmodel.h5')  # creates a HDF5 file 'my_model.h5'
#del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
#modelkk = load_model('mmmodel.h5')

Train on 1711 samples, validate on 191 samples
Epoch 1/100
1711/1711 [==============================] - 34s 20ms/step - loss: 0.4440 - acc: 0.9317 - val_loss: 0.2498 - val_acc: 0.9479
Epoch 2/100
1711/1711 [==============================] - 20s 12ms/step - loss: 0.2196 - acc: 0.9488 - val_loss: 0.2079 - val_acc: 0.9477
Epoch 3/100
1711/1711 [==============================] - 21s 12ms/step - loss: 0.1582 - acc: 0.9560 - val_loss: 0.1404 - val_acc: 0.9604
Epoch 4/100
1711/1711 [==============================] - 22s 13ms/step - loss: 0.1135 - acc: 0.9670 - val_loss: 0.1132 - val_acc: 0.9666
Epoch 5/100
1711/1711 [==============================] - 21s 12ms/step - loss: 0.0911 - acc: 0.9736 - val_loss: 0.0962 - val_acc: 0.9723
Epoch 6/100
1711/1711 [==============================] - 18s 11ms/step - loss: 0.0738 - acc: 0.9795 - val_loss: 0.0826 - val_acc: 0.9768
Epoch 7/100
1711/1711 [==============================] - 19s 11ms/step - loss: 0.0583 - acc: 0.9842 - val_loss: 0.0707 - val_acc: 0

1711/1711 [==============================] - 19s 11ms/step - loss: 2.7021e-04 - acc: 0.9999 - val_loss: 0.0306 - val_acc: 0.9958
Epoch 60/100
1711/1711 [==============================] - 19s 11ms/step - loss: 3.0065e-04 - acc: 0.9999 - val_loss: 0.0303 - val_acc: 0.9955
Epoch 61/100
1711/1711 [==============================] - 20s 12ms/step - loss: 2.1483e-04 - acc: 0.9999 - val_loss: 0.0294 - val_acc: 0.9961
Epoch 62/100
1711/1711 [==============================] - 19s 11ms/step - loss: 4.0190e-04 - acc: 0.9999 - val_loss: 0.0329 - val_acc: 0.9953
Epoch 63/100
1711/1711 [==============================] - 19s 11ms/step - loss: 3.5149e-04 - acc: 0.9999 - val_loss: 0.0315 - val_acc: 0.9956
Epoch 64/100
1711/1711 [==============================] - 19s 11ms/step - loss: 2.8292e-04 - acc: 0.9999 - val_loss: 0.0301 - val_acc: 0.9957
Epoch 65/100
1711/1711 [==============================] - 18s 11ms/step - loss: 3.3766e-04 - acc: 0.9999 - val_loss: 0.0312 - val_acc: 0.9957
Epoch 66/100
1711/1

In [7]:
test_sentence = "An individual wiping command should be implemented by setting the level of WW and INT-line from HIGH to LOW for the parameterized time p_t_einzelwischhub body control module".split(" ")

In [8]:
x_test_sent = pad_sequences(sequences=[[word2idx.get(w, 0) for w in test_sentence]],
                            padding="post", value=0, maxlen=max_len)

In [9]:
def newpredict(tt):
    result = []
    x_test_sent = pad_sequences(sequences=[[word2idx.get(w, 0) for w in tt]],
                            padding="post", value=0, maxlen=max_len)
    p = model.predict(np.array([x_test_sent[0]]))
    p = np.argmax(p, axis=-1)
    for w, pred in zip(tt, p[0]):
        result.append((w, tags[pred]))
    return result

In [10]:
newpredict(test_sentence)

[('An', 'O'),
 ('individual', 'O'),
 ('wiping', 'O'),
 ('command', 'O'),
 ('should', 'O'),
 ('be', 'O'),
 ('implemented', 'O'),
 ('by', 'O'),
 ('setting', 'O'),
 ('the', 'O'),
 ('level', 'O'),
 ('of', 'O'),
 ('WW', 'O'),
 ('and', 'O'),
 ('INT-line', 'O'),
 ('from', 'O'),
 ('HIGH', 'O'),
 ('to', 'O'),
 ('LOW', 'O'),
 ('for', 'O'),
 ('the', 'O'),
 ('parameterized', 'O'),
 ('time', 'O'),
 ('p_t_einzelwischhub', 'O'),
 ('body', 'O'),
 ('control', 'O'),
 ('module', 'O')]

In [11]:
[('The', 'O'),
 ('system', 'Network_Component'),
 ('should', 'O'),
 ('do', 'O'),
 ('the', 'O'),
 ('validation', 'Action'),
 ('for', 'O'),
 ('confirm', 'Action'),
 ('password', 'Attribute'),
 ('text', 'O'),
 ('on', 'O'),
 ('submit', 'O'),
 ('of', 'O'),
 ('the', 'O'),
 ('form', 'O')]

[('The', 'O'),
 ('system', 'Network_Component'),
 ('should', 'O'),
 ('do', 'O'),
 ('the', 'O'),
 ('validation', 'Action'),
 ('for', 'O'),
 ('confirm', 'Action'),
 ('password', 'Attribute'),
 ('text', 'O'),
 ('on', 'O'),
 ('submit', 'O'),
 ('of', 'O'),
 ('the', 'O'),
 ('form', 'O')]

In [12]:
scores = model.evaluate(X_tr, np.array(y_tr), verbose=1)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

1902/1902 [==============================] - 5s 2ms/step
acc: 99.95%


In [13]:
# serialize model to JSON
model_json = model.to_json()
with open("model2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model1.h5")
print("Saved model to disk")

Saved model to disk


In [14]:
from keras.models import model_from_json
# load json and create model
json_file = open('model2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model1.h5")
print("Loaded model from disk")

Loaded model from disk


In [15]:
import os
os.getcwd()

'C:\\Users\\hp\\python programs in anaconda class notes\\FOr ESDS Github\\NER-Named-Entity-Recognition-master'

In [16]:
loaded_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [17]:
score = loaded_model.evaluate(X_tr, np.array(y_tr), verbose=1)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

1902/1902 [==============================] - 8s 4ms/step
acc: 99.95%


In [18]:
import numpy as np

# Save
np.save('W2i.npy', word2idx) 

# Load
#word2idx = np.load('C:\\Users\\sharath\\Desktop\\exp\\W2i.npy').item()


# Save
np.save('T2i.npy', tag2idx) 

# Load
#tag2idx = np.load('C:\\Users\\sharath\\Desktop\\exp\\T2i.npy').item()

In [19]:
def bilstmfinal(test_sentence):
    import pandas as pd
    import numpy as np
    from keras.models import model_from_json
    from keras.preprocessing.sequence import pad_sequences
    from keras.models import Model, Input
    from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
    from sklearn.model_selection import train_test_split
    from keras.utils import to_categorical
    import numpy
    numpy.random.seed(7)
    word2idx = np.load('W2i.npy').item()
    tag2idx = np.load('T2i.npy').item()
    # load json and create model
    json_file = open('model2.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("model1.h5")
    loaded_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    test_sentence = test_sentence.split(" ")
    x_test_sent = pad_sequences(sequences=[[word2idx.get(w, 0) for w in test_sentence]],
                            padding="post", value=0, maxlen=50)
    p = loaded_model.predict(np.array([x_test_sent[0]]))
    p = np.argmax(p, axis=-1)
    pls = []
    p = loaded_model.predict(np.array([x_test_sent[0]]))
    p = np.argmax(p, axis=-1)
    for w, pred in zip(test_sentence, p[0]):
        pls.append(pred)
    revsubs = { v:k for k,v in tag2idx.items()}
    entit = [revsubs.get(item,item) for item in pls]
    final = list(zip(test_sentence,entit))
    return final

In [20]:
s = "The system shall allow a registered visitor to load a custom filter."

In [21]:
bilstmfinal(s)

[('The', 'O'),
 ('system', 'O'),
 ('shall', 'O'),
 ('allow', 'O'),
 ('a', 'O'),
 ('registered', 'actor'),
 ('visitor', 'actor'),
 ('to', 'O'),
 ('load', 'action'),
 ('a', 'O'),
 ('custom', 'attribute'),
 ('filter.', 'attribute')]